In [98]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from math import sqrt

In [117]:
pisa_train = pd.read_csv('./data/pisa2009train.csv')
pisa_test = pd.read_csv('./data/pisa2009test.csv')

In [118]:
# number of students
len(pisa_train)

3663

In [119]:
# what is the average reading test score of males and females?
pisa_train.pivot_table(values='readingScore', columns='male', aggfunc=np.mean)

male
0    512.940631
1    483.532479
Name: readingScore, dtype: float64

In [120]:
# Which variables are missing data in at least one observation in the training set?
# Could be accomplished with pisa_train.summary()

In [121]:
# Linear regression discards observations with missing data,
# so we will remove all such observations from the training and testing sets
pisa_train = pisa_train.dropna()
pisa_test = pisa_test.dropna()

# number of observations without missing values
print(len(pisa_train))
print(len(pisa_test))

2414
990


In [122]:
# Building a model
# Because the race variable takes on text values, it was loaded as a factor variable when we read in the dataset.
# To include unordered factors in a linear regression model, we define one level as the "reference level"
# and add a binary variable for each of the remaining levels.
# In this way, a factor with n levels is replaced by n-1 binary variables.
# The reference level is typically selected to be the most frequently occurring level in the dataset.
# We can do this by passing our reference variable to patsy.Treatment (error with test data)

# variable += ' + C(pisa_train.raceeth, Treatment(reference="White"))'
renamed = {'American Indian/Alaska Native': 'AmericanIndianAlaskaNative',
           'Native Hawaiian/Other Pacific Islander': 'NativeHawaiianOtherPacificIslander',
           'More than one race': 'Morethanonerace'}

dummies = pd.get_dummies(pisa_train.raceeth)
pisa_train = pd.concat([pisa_train, dummies], axis=1)
pisa_train = pisa_train.drop(['raceeth', 'White'], axis=1)
pisa_train.rename(columns=renamed, inplace=True)

dummies = pd.get_dummies(pisa_test.raceeth)
pisa_test = pd.concat([pisa_test, dummies], axis=1)
pisa_test = pisa_test.drop(['raceeth', 'White'], axis=1)
pisa_test.rename(columns=renamed, inplace=True)

variable = list(pisa_train.columns)
variable.remove('readingScore')
variable = ' + '.join(variable)

pisa_reg = sm.ols(formula='readingScore ~ '+variable, data=pisa_train).fit()
pisa_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           readingScore   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     41.04
Date:                Thu, 30 Jun 2016   Prob (F-statistic):          1.72e-180
Time:                        19:43:05   Log-Likelihood:                -13795.
No. Observations:                2414   AIC:                         2.765e+04
Df Residuals:                    2385   BIC:                         2.781e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------------
Intercept                            143.7663     33.841      4.248      0.000        77.405   210.128
grade                                 29.5427      2.937     10.057      0.000        23.783    35.303
male                                 -14.5217      3.156     -4.601      0.000       -20.710    -8.333
preschool                             -4.4637      3.486     -1.280      0.201       -11.300     2.372
expectBachelors                       55.2671      4.294     12.871      0.000        46.847    63.687
motherHS                               6.0588      6.091      0.995      0.320        -5.886    18.004
motherBachelors                       12.6381      3.861      3.273      0.001         5.066    20.210
motherWork                            -2.8091      3.522     -0.798      0.425        -9.715     4.097
fatherHS                               4.0182      5.579      0.720      0.471        -6.923    14.959
fatherBachelors                       16.9298      3.995      4.237      0.000         9.095    24.764
fatherWork                             5.8428      4.396      1.329      0.184        -2.778    14.463
selfBornUS                            -3.8063      7.324     -0.520      0.603       -18.168    10.555
motherBornUS                          -8.7982      6.588     -1.336      0.182       -21.716     4.120
fatherBornUS                           4.3070      6.264      0.688      0.492        -7.976    16.590
englishAtHome                          8.0357      6.859      1.171      0.242        -5.415    21.487
computerForSchoolwork                 22.5002      5.703      3.946      0.000        11.318    33.683
read30MinsADay                        34.8719      3.408     10.231      0.000        28.188    41.556
minutesPerWeekEnglish                  0.0128      0.011      1.194      0.233        -0.008     0.034
studentsInEnglish                     -0.2866      0.228     -1.258      0.208        -0.733     0.160
schoolHasLibrary                      12.2151      9.265      1.318      0.187        -5.953    30.383
publicSchool                         -16.8575      6.726     -2.506      0.012       -30.046    -3.669
urban                                 -0.1101      3.963     -0.028      0.978        -7.881     7.661
schoolSize                             0.0065      0.002      2.977      0.003         0.002     0.011
AmericanIndianAlaskaNative           -67.2773     16.787     -4.008      0.000      -100.196   -34.359
Asian                                 -4.1103      9.220     -0.446      0.656       -22.191    13.970
Black                                -67.0123      5.461    -12.271      0.000       -77.721   -56.304
Hispanic                             -38.9755      5.178     -7.528      0.000       -49.129   -

In [123]:
# R2 = 0.325

Note that this R-squared is lower than the ones for the models we saw in the lectures and recitation. This does not necessarily imply that the model is of poor quality. More often than not, it simply means that the prediction problem at hand (predicting a student's test score based on demographic and school-related variables) is more difficult than other prediction problems (like predicting a team's number of wins from their runs scored and allowed, or predicting the quality of wine from weather conditions).

In [124]:
# Computing the root-mean squared error of the model
SSE = sum((pisa_reg.resid)**2)
RMSE = sqrt(SSE/len(pisa_train))
RMSE

73.36555143298459

In [127]:
# Predicting on unseen data
# What is the range between the maximum and minimum predicted reading score on the test set?

predictions = pisa_reg.predict(pisa_test)
predictions.max() - predictions.min()

284.46831179512253

In [131]:
# Test set SSE and RMSE
SSE = sum((predictions - pisa_test.readingScore)**2)
RMSE = sqrt(SSE/len(pisa_test))
RMSE

76.29079383109166

In [132]:
# Baseline prediction and test-set SSE
baseline = np.mean(pisa_train.readingScore)
baseline

517.96288732394362

In [134]:
# What is the sum of squared errors of the baseline model on the testing set?
SST = sum((baseline - pisa_test.readingScore)**2)
SST

7802354.0776138408

In [135]:
# Test-set R-squared
R2 = 1 - SSE/SST
R2

0.26149437543770848